In [1]:
import tensorflow as tf
import numpy as np
import math
import time
import os
tf.__version__

'1.8.0'

In [2]:
data_dir = "C:/Users/asd/Desktop/Winter Training/BootcampStudents/Data/Task_1/"
# log_dir = "C:/Users/asd/Desktop/Winter Training/BootcampStudents/log/"

In [3]:
def load_data(kind):
    """
    Akind -  ANN / LSTM
    """
    data = np.load(os.path.join(data_dir, "%s.npz"%kind))
    train_x, train_y = data["train_x"], data["train_y"]
    val_x, val_y = data["val_x"], data["val_y"]
    return train_x, train_y, val_x, val_y

In [4]:
trainData,trainLabel,valData,valLabel=load_data('LSTM')

In [5]:
print(trainData.shape)
print(trainLabel.shape)
print(valData.shape)
print(valLabel.shape)

(588, 50, 2048)
(588, 5)
(58, 50, 2048)
(58, 5)


In [6]:
def create_batches(start,batchSize):
    return trainData[start:start+batchSize],trainLabel[start:start+batchSize]

In [7]:
# train,label=create_batches(0,50)
# print(np.shape(train))


### ModelGraph

We first define the two place holders- one for inputs and the other for the labels. The input to the model are 50 videos, each having 2048 features while the output must classify them as in 5 different classes of videos. The inputs are fed to the RNN which has 128 neurons when rolled out. Each neuron has its own output and the state. 

In [8]:
x=tf.placeholder(tf.float32,shape=[None,50, 2048])
y_ = tf.placeholder(tf.float32, shape=[None,5])

We define the cell as a basic RNN cell having 128 neurons, each having its own output and states. The states being redundant in our case, have been deleted to not take up the extra memory.

In [9]:
cell=tf.nn.rnn_cell.BasicRNNCell(128)

The outputs and state are found using tf.nn.dynamic_rnn(cell,input). This means the visual input is fed to the RNNs directly first. The 128 neurons in the RNN sives their outputs. These outputs arethen fed to the ANN layer.

In [10]:
outputs, state = tf.nn.dynamic_rnn(cell, x ,dtype=tf.float32)
del state

The output is flattened to be fed to the neurons in the first Artificial Neural Network which forms the result according to them.

In [11]:
output=tf.reshape(outputs,[-1,50*128])

In [12]:
def initializeLayer(inputs,outputs):
    return tf.Variable(tf.truncated_normal([inputs,outputs],stddev=0.1)), tf.Variable(tf.zeros([outputs]))#input to output dimensions of the weight matrix

### Calculating the cross entropy

In [13]:
def lossCompute():
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

### Training Op

In [14]:
def trainstep(alpha, crossentropy):
    return tf.train.GradientDescentOptimizer(alpha).minimize(crossentropy)

In [15]:
def findaccuracy(y,y_):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return correct_prediction,accuracy

In [16]:
# def compute(epochs,iterations):
#     for i in range(epochs):
#         for j in range(iterations):
#             batchData,batchLabel=create_batches(j*98,98,0)
#             train_step.run(feed_dict={x: batchData, y_: batchLabel})
#         print("accuracy", accuracy.eval(feed_dict={x: valData, y_: valLabel}))

In [ ]:
with tf.Session() as sess:
    W1,B1=initializeLayer(50*128,128)
    W2,B2=initializeLayer(128,5)
    sess.run(tf.global_variables_initializer())
    #x = tf.placeholder(tf.float32, shape=[None,102400])
    #y_ = tf.placeholder(tf.float32, shape=[None, 5])
    y1 = tf.matmul(output,W1) + B1
    fy1 = tf.nn.leaky_relu(y1)
    y = tf.matmul(y1,W2) + B2
    cross_entropy=lossCompute()
    train_step = trainstep(0.7e-3,cross_entropy)
    correct_prediction,accuracy=findaccuracy(y,y_)
    for i in range(20):
        for j in range(6):
            batchData,batchLabel=create_batches(j*98,98)
            train_step.run(feed_dict={x: batchData, y_: batchLabel})
        print("accuracy", accuracy.eval(feed_dict={x: valData, y_: valLabel}))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

accuracy 0.258621
accuracy 0.482759
accuracy 0.586207
accuracy 0.586207
accuracy 0.603448
accuracy 0.637931
